### Setup ###

In [57]:
import mysql.connector
import pandas as pd

# Create connection
conn = mysql.connector.connect(
    host="localhost",
    user="root",           # Your MySQL username
    password="40418",  # Your MySQL password
    database="aeras"       # Your database name
)

print("✅ Connected!")

✅ Connected!


In [58]:
cursor = conn.cursor()

In [59]:
df = pd.read_sql("SELECT * FROM users", conn)
print(df)

Empty DataFrame
Columns: [id, userid, name, privilege, flag, ban, password, sign_up, phone]
Index: []


C:\Users\dibya\AppData\Local\Temp\ipykernel_16756\3329270977.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM users", conn)


In [60]:
import hashlib

def encoding(input_string):
    sha256_hash = hashlib.sha256()
    sha256_hash.update(input_string.encode('utf-8'))
    return sha256_hash.hexdigest()

In [61]:
def genID(iD,signup,phone):
    Id=signup[2:4]+phone[4:6]+str(iD).zfill(5)
    return Id

### User details ###

In [62]:
cursor.execute("SELECT * FROM users")
iD=len(cursor.fetchall())+1
name="Abul Karim"
privilege="underprevileged"
flag=0
ban=0
password="12345"
signup="2025-11-13"
phone="01312731509"

In [63]:
insert_query = """
INSERT INTO users (userid, name, privilege, flag, ban, password, sign_up, phone) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""
data = (encoding(genID(iD,signup,phone)), encoding(name), privilege, flag, ban, encoding(password), signup, encoding(phone))

cursor.execute(insert_query, data)
conn.commit()

print(f"✅ Added {cursor.rowcount} user")

✅ Added 1 user


In [64]:
df = pd.read_sql("SELECT * FROM users", conn)
print(df)

   id                                             userid  \
0   1  5587b24e1a7995ff2ac3be95fbce707e78f87752068226...   

                                                name        privilege  flag  \
0  9c4208cb4efed93976bef30f22e3cc69910461ccffa116...  underprevileged     0   

   ban                                           password     sign_up  \
0    0  5994471abb01112afcc18159f6cc74b4f511b99806da59...  2025-11-13   

                                               phone  
0  31501087a442c31f3d765791281a0028dad21fdbdce872...  


C:\Users\dibya\AppData\Local\Temp\ipykernel_16756\3329270977.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM users", conn)


## Don't use DELETE command ###

In [65]:
cursor.execute("TRUNCATE TABLE users")
conn.commit()
df = pd.read_sql("SELECT * FROM users", conn)
print(df)

Empty DataFrame
Columns: [id, userid, name, privilege, flag, ban, password, sign_up, phone]
Index: []


C:\Users\dibya\AppData\Local\Temp\ipykernel_16756\3490582095.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM users", conn)
